In [1]:
import torch
import numpy as np
import torchvision.transforms as T
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image


In [2]:
# Mapping von Mapillary Vistas IDs auf Cityscapes IDs (detaillierter)
mapillary_to_cityscapes = {
    # Road-related Klassen
    21: 0,   # Road -> Road
    16: 0,   # Driveway -> Road
    17: 0,   # Parking -> Road
    18: 0,   # Parking Aisle -> Road
    22: 0,   # Road Shoulder -> Road
    23: 0,   # Service Lane -> Road

    # Sidewalk
    24: 1,   # Sidewalk -> Sidewalk
    19: 1,   # Pedestrian Area -> Sidewalk
    4: 1,    # Curb -> Sidewalk

    # Building-related Klassen
    27: 2,   # Building -> Building
    28: 2,   # Garage -> Building
    29: 2,   # Tunnel -> Building
    26: 2,   # Bridge -> Building

    # Wall
    12: 3,   # Wall -> Wall

    # Fence-related Klassen
    5: 4,    # Fence -> Fence
    6: 4,    # Guard Rail -> Fence
    7: 4,    # Barrier -> Fence
    2: 4,    # Ambiguous Barrier -> Fence
    3: 4,    # Concrete Block -> Fence
    8: 4,    # Road Median -> Fence
    9: 4,    # Road Side -> Fence
    11: 4,   # Temporary Barrier -> Fence

    # Poles
    85: 5,   # Pole -> Pole
    86: 5,   # Pole Group -> Pole
    88: 5,   # Utility Pole -> Pole

    # Traffic Lights
    90: 6,   # Traffic Light - General -> Traffic Light
    91: 6,   # Traffic Light - Pedestrians -> Traffic Light
    92: 6,   # Traffic Light - Upright -> Traffic Light
    93: 6,   # Traffic Light - Horizontal -> Traffic Light
    94: 6,   # Traffic Light - Cyclists -> Traffic Light
    95: 6,   # Traffic Light - Other -> Traffic Light

    # Traffic Signs
    99: 7,   # Traffic Sign (Front) -> Traffic Sign
    100: 7,  # Traffic Sign (Front) -> Traffic Sign
    103: 7,  # Traffic Sign - Temporary (Front) -> Traffic Sign

    # Vegetation
    64: 8,   # Vegetation -> Vegetation

    # Terrain
    63: 9,   # Terrain -> Terrain
    59: 9,   # Mountain -> Terrain
    60: 9,   # Sand -> Terrain
    62: 9,   # Snow -> Terrain

    # Sky
    61: 10,  # Sky -> Sky

    # Person
    30: 11,  # Person -> Person
    31: 11,  # Person Group -> Person

    # Riders (Fahrer)
    32: 12,  # Bicyclist -> Rider
    33: 12,  # Motorcyclist -> Rider
    34: 12,  # Other Rider -> Rider

    # Vehicles
    108: 13, # Car -> Car
    115: 13, # Vehicle Group -> Car

    114: 14, # Truck -> Truck
    113: 14, # Trailer -> Truck

    107: 15, # Bus -> Bus

    111: 16, # On Rails -> Train

    110: 17, # Motorcycle -> Motorcycle

    105: 18, # Bicycle -> Bicycle

    # Alles andere als "Unlabeled"
    123: 19, # Unlabeled -> Unlabeled
}

# Falls eine Klasse nicht gemappt wurde, soll sie als "unlabeled" behandelt werden
default_cityscapes_label = 19


In [3]:
# Detailliertes Mapping von Mapillary Vistas IDs auf Cityscapes IDs für das 03-mini-high Modell.
# Hier wird sichergestellt, dass auch feine Unterteilungen, wie sie in Mapillary Vistas vorkommen,
# zu den übergeordneten Cityscapes-Klassen zusammengefasst werden.

mapillary_to_cityscapes_mini = {
    # Road: Hier werden verschiedene Straßenelemente zusammengefasst.
    21: 0,   # Road -> Road
    16: 0,   # Driveway -> Road
    17: 0,   # Parking -> Road
    18: 0,   # Parking Aisle -> Road
    22: 0,   # Road Shoulder -> Road
    23: 0,   # Service Lane -> Road

    # Sidewalk: Unterschiedliche Formen von Gehwegen und Fußgängerbereichen
    24: 1,   # Sidewalk -> Sidewalk
    19: 1,   # Pedestrian Area -> Sidewalk
    4: 1,    # Curb -> Sidewalk

    # Building: Gebäude, Garagen, Tunnel und Brücken werden als Bauwerk zusammengefasst.
    27: 2,   # Building -> Building
    28: 2,   # Garage -> Building
    29: 2,   # Tunnel -> Building
    26: 2,   # Bridge -> Building

    # Wall: Fassade bzw. Wand
    12: 3,   # Wall -> Wall

    # Fence: Verschiedene Barrieren und Zäune
    5: 4,    # Fence -> Fence
    6: 4,    # Guard Rail -> Fence
    7: 4,    # Barrier -> Fence
    2: 4,    # Ambiguous Barrier -> Fence
    3: 4,    # Concrete Block -> Fence
    8: 4,    # Road Median -> Fence
    9: 4,    # Road Side -> Fence
    11: 4,   # Temporary Barrier -> Fence

    # Pole: Säulen und ähnliche Strukturen
    85: 5,   # Pole -> Pole
    86: 5,   # Pole Group -> Pole
    88: 5,   # Utility Pole -> Pole

    # Traffic Light: Verschiedene Varianten von Ampeln
    90: 6,   # Traffic Light - General -> Traffic Light
    91: 6,   # Traffic Light - Pedestrians -> Traffic Light
    92: 6,   # Traffic Light - Upright -> Traffic Light
    93: 6,   # Traffic Light - Horizontal -> Traffic Light
    94: 6,   # Traffic Light - Cyclists -> Traffic Light
    95: 6,   # Traffic Light - Other -> Traffic Light

    # Traffic Sign: Verschiedene Verkehrszeichen
    99: 7,   # Traffic Sign (Front) -> Traffic Sign
    100: 7,  # Traffic Sign (Front) -> Traffic Sign
    103: 7,  # Traffic Sign - Temporary (Front) -> Traffic Sign

    # Vegetation
    64: 8,   # Vegetation -> Vegetation

    # Terrain: Diverse Bodentypen und Landschaftselemente
    63: 9,   # Terrain -> Terrain
    59: 9,   # Mountain -> Terrain
    60: 9,   # Sand -> Terrain
    62: 9,   # Snow -> Terrain

    # Sky
    61: 10,  # Sky -> Sky

    # Person: Einzelpersonen oder Gruppen
    30: 11,  # Person -> Person
    31: 11,  # Person Group -> Person

    # Rider: Unterschiedliche Formen von Fahrern
    32: 12,  # Bicyclist -> Rider
    33: 12,  # Motorcyclist -> Rider
    34: 12,  # Other Rider -> Rider

    # Car: Pkws und Fahrzeuggruppen, die als Auto interpretiert werden
    108: 13, # Car -> Car
    115: 13, # Vehicle Group -> Car

    # Truck: Lastwagen und Anhänger
    114: 14, # Truck -> Truck
    113: 14, # Trailer -> Truck

    # Bus
    107: 15, # Bus -> Bus

    # Train: Schienenfahrzeuge
    111: 16, # On Rails -> Train

    # Motorcycle
    110: 17, # Motorcycle -> Motorcycle

    # Bicycle
    105: 18, # Bicycle -> Bicycle

    # Unlabeled: Alle nicht zugeordneten Klassen
    123: 19, # Unlabeled -> Unlabeled
}


In [5]:
print( mapillary_to_cityscapes_mini == mapillary_to_cityscapes)

True
